In [1]:
from datetime import datetime
from urllib.request import urlopen
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
url="https://finance.yahoo.com/quote/AAPL/financials"

In [16]:
def fundamentals_from_yahoo(ticker):
    url="https://finance.yahoo.com/quote/%s/financials" % ticker
    soup = BeautifulSoup(urlopen(url))
    rows = soup.find_all(text='Gross Profit')[0].parent.parent.parent.parent.contents
    
    fundis=[]
    for row in rows:
        if len(row.contents) > 3:
            try:
                key = row.contents[0].contents[0].text
                if key == 'Revenue':
                    fundis.append(["Fundamental"] + 
                                  [datetime.strptime(col.contents[0].text, '%m/%d/%Y').date()
                                   for col in row.contents[1:]])
                else:
                    fundis.append([key] + [
                        int(col.contents[0].text.replace(',',''))/1e6 
                        for col in row.contents[1:]])
            except:
                pass

    df = pd.DataFrame(fundis[1:], columns=fundis[0])
    return df

In [18]:
fundamentals_from_yahoo('NFLX')

,Fundamental,2018-12-31,2017-12-31,2016-12-31,2015-12-31
0,Total Revenue,15.794341,11.692713,8.830669,6.779511
1,Cost of Revenue,9.967538,8.033000,6.257462,4.591476
2,Gross Profit,5.826803,3.659713,2.573207,2.188035
3,Research Development,1.221814,0.953710,0.780232,0.650788
4,Selling General and Administrative,2.999763,1.867324,1.413182,1.231421
5,Total Operating Expenses,14.189115,10.854034,8.450876,6.473685
6,Operating Income or Loss,1.605226,0.838679,0.379793,0.305826
7,Total Other Income/Expenses Net,-0.378768,-0.353358,-0.119286,-0.163941
8,Earnings Before Interest and Taxes,1.605226,0.838679,0.379793,0.305826
9,Interest Expense,-0.420493,-0.238204,-0.150114,-0.132716
